**Generating O_D matrix and Trip Generation and Attraction matrices using Geohash method**

In [ ]:
#Libraries
!pip install pygeohash
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
import branca.colormap as cm
import pygeohash as gh

#method arguments 
PRESISION = 6
hour_intevral = 1
day_interval = 24

**Load data**

In [ ]:
filename = '../input/nyc-taxi-trip-duration/train.zip'
data = pd.read_csv(filename)

data.head()

**Generate O-D matrix**

In [ ]:
origin_hashes=data.apply(lambda x: gh.encode(x.pickup_latitude, x.pickup_longitude, PRESISION), axis=1)
destination_hashes=data.apply(lambda x: gh.encode(x.dropoff_latitude, x.dropoff_longitude, PRESISION), axis=1)

date = data['pickup_datetime'].values
intervals = date.astype('datetime64').astype('int')//(24*3600) 

n_days = np.unique(intervals).shape[0]
df = pd.DataFrame({'interval':intervals, 
                   'origin':origin_hashes, 
                   'destination':destination_hashes})

daily_od = df.groupby(['interval','origin','destination']).size().reset_index(name='counts')
ave_daily_od = daily_od.groupby(['origin','destination']).agg({'counts':lambda x: np.sum(x)/n_days}).reset_index()

ave_daily_od.head()

**Visualize OD matrix**

In [ ]:
min_trip = 0
max_trip = int(ave_daily_od['counts'].values.max())+1
colormap = cm.linear.Blues_06.scale(vmin = min_trip,vmax = max_trip)

mymap = folium.Map(width=800,height=500, location=[40.843761,-73.871846], zoom_start=10)

for row in ave_daily_od.iloc:
    origin_hash =  row['origin']
    decoded_origin =  gh.decode_exactly(origin_hash)
    destination_hash =  row['destination']
    destination_origin =  gh.decode_exactly(destination_hash)
    line = [[decoded_origin[0], decoded_origin[1]],
            [destination_origin[0],destination_origin[1]]]
    trip = row['counts']
    
    folium.PolyLine(line,clustered_marker = True, 
                    color='blue', 
                    opacity = trip/max_trip).add_to(mymap)

mymap.add_child(colormap)
display(mymap)

**Trip Generation**

In [ ]:
trip_generation = ave_daily_od.groupby(['origin']).agg({'counts': 'sum'}).reset_index()

mymap = folium.Map(width=800,height=500, location=[40.843761,-73.871846], zoom_start=10)
inds = [0, 5, 12, 25, 50, 100, 200, 300]
colormap = cm.StepColormap(colors = ['blue', 'royalblue', 'yellow','orange', 'darkorange','red', 'firebrick'], 
                           index=inds, vmin = trip_generation.counts.min(), 
                           vmax = trip_generation.counts.max())
colormap.caption = 'Trip generation(No. trips per day)'

for row in trip_generation.iloc:
    g_hash =  row['origin']
    n_trip = row['counts']
    decoded_hash =  gh.decode_exactly(g_hash)

    x, y =decoded_hash[0], decoded_hash[1]
    delta_x = decoded_hash[2]
    delta_y = decoded_hash[3]
    rec = [[x-delta_x,y-delta_y],
           [x+delta_x,y-delta_y],
           [x+delta_x,y+delta_y],
           [x-delta_x,y+delta_y]]
    folium.Polygon(rec, fill_color=colormap(n_trip),fill_opacity=0.7,
                   color='white',weight=0.5,
                   tooltip="{}: {}".format(g_hash,n_trip)).add_to(mymap)


mymap.add_child(colormap)
display(mymap)

**Trip Attraction**

In [ ]:
trip_attraction = ave_daily_od.groupby(['destination']).agg({'counts': 'sum'}).reset_index()

mymap = folium.Map(width=800,height=500, location=[40.843761,-73.871846], zoom_start=10)
inds = [0, 5, 12, 25, 50, 100, 200, 300]
colormap = cm.StepColormap(colors = ['blue', 'royalblue', 'yellow','orange', 'darkorange','red', 'firebrick'], 
                           index=inds, vmin = trip_attraction.counts.min(), 
                           vmax = trip_attraction.counts.max())
colormap.caption = 'Trip Attraction(No. trips per day)'

for row in trip_attraction.iloc:
    g_hash =  row['destination']
    n_trip = row['counts']
    decoded_hash =  gh.decode_exactly(g_hash)

    x, y =decoded_hash[0], decoded_hash[1]
    delta_x = decoded_hash[2]
    delta_y = decoded_hash[3]
    rec = [[x-delta_x,y-delta_y],
           [x+delta_x,y-delta_y],
           [x+delta_x,y+delta_y],
           [x-delta_x,y+delta_y]]
    folium.Polygon(rec, fill_color=colormap(n_trip),fill_opacity=0.7,
                   color='white',weight=0.5,
                   tooltip="{}: {}".format(g_hash,n_trip)).add_to(mymap)


mymap.add_child(colormap)
display(mymap)